# go-常用命令

* go version: 查看go的版本
* go run hello.go: 编译运行程序
* go build hello.go: 编译包和依赖生成可执行文件,windows上生成*.exe文件
* go clean hello.go: 调用 clean 后会删除编译生成的可执行文件。
* go vet main.go: go vet 工具不能让开发者避免严重的逻辑错误，或者避免编写充满小错的代码。不过，这个工具可以很好地捕获一部分常见错误。每次对代码先执行 go vet 再将其签入源代码库是一个很好的习惯。
* go fmt 后面跟文件名或者包名，就可以调用这个代码格式化工具。fmt 命令会自动格式化开发人员指定的源代码文件并保存。
* godoc: 终端上直接使用 go doc 命令来打印文档
* godoc tar: 用 Go 语言第一次开发读取 UNIX tar 文件的应用程序，想要看看 archive/tar 包的相关文档
* godoc -http:=6060 在浏览器打开`http://localhost:6060`,即可看到go的官方文档
* go get 

### go build

这个命令主要用于编译代码。在包的编译过程中，若有必要，会同时编译与之相关联的包。

* 普通包(应用包): 执行go build之后，不会产生任何文件。如果需要在`$GOPATH/pkg`下生成相应的文件，那就得执行`go install`。
* main包: 当执行go build之后，就会在当前目录下生成一个可执行文件。如果需要在`$GOPATH/bin`下生成相应的文件，需要执行`go install`，或者使用`go build -o 路径/a.exe`。
* 只想编译某个文件，就在`go build`之后加上文件名，如`go build a.go`, go build命令默认会编译当前目录下的所有go文件。
* 指定编译输出的文件名。指定`go build -o astaxie.exe`，默认情况是package名(非main包)，或第一个源文件的文件名(main包)。
* go build会忽略目录下以“_”或“.”开头的go文件。
* 如果你的源代码对不同的操作系统需要不同的处理，那么可以根据不同的操作系统后缀来命名文件。例如有一个读取数组的程序，它对于不同的操作系统可能有如下几个源文件：`array_linux.go array_darwin.go array_windows.go array_freebsd.go`,go build时会选择性地编译以系统名结尾的文件（Linux、Darwin、Windows、Freebsd）。例如Linux系统下面编译只会选择`array_linux.go`文件，其它系统命名后缀文件全部忽略。

### go clean

这个命令是用来移除当前源码包和关联源码包里面编译生成的文件。这些文件包括
```
_obj/            旧的object目录，由Makefiles遗留
_test/           旧的test目录，由Makefiles遗留
_testmain.go     旧的gotest文件，由Makefiles遗留
test.out         旧的test记录，由Makefiles遗留
build.out        旧的test记录，由Makefiles遗留
*.[568ao]        object文件，由Makefiles遗留

DIR(.exe)        由go build产生
DIR.test(.exe)   由go test -c产生
MAINFILE(.exe)   由go build MAINFILE.go产生
*.so             由 SWIG 产生
```

参数介绍

* -i 清除关联的安装的包和可运行文件，也就是通过go install安装的文件
* -n 把需要执行的清除命令打印出来，但是不执行，这样就可以很容易的知道底层是如何运行的
* -r 循环的清除在import中引入的包
* -x 打印出来执行的详细命令，其实就是-n打印的执行版本

### go fmt

你只需要在写完之后执行`go fmt <文件名>.go`，你的代码就被修改成了标准格式，

开发工具一般都带了保存时自动格式化功能，这个功能其实在底层调用了go fmt。

使用go fmt命令,其实是调用`gofmt`，需要参数`-w`，否则格式化结果不会写入文件。

`gofmt -w -l src`，可以格式化整个项目。所以go fmt是gofmt的一个上层包装命令.

gofmt的参数介绍

* -l 显示那些需要格式化的文件
* -w 把改写后的内容直接写入到文件中，而不是作为结果打印到标准输出。
* -r 添加形如“a[b:len(a)] -> a[b:]”的重写规则，方便我们做批量替换
* -s 简化文件中的代码
* -d 显示格式化前后的diff而不是写入文件，默认是false
* -e 打印所有的语法错误到标准输出。如果不使用此标记，则只会打印不同行的前10个错误。
* -cpuprofile 支持调试模式，写入相应的cpufile到指定的文件

### go get

这个命令在内部实际上分成了两步操作：第一步是下载源码包，第二步是执行go install。

下载源码包的go工具会自动根据不同的域名调用不同的源码工具，对应关系如下：
```
BitBucket (Mercurial Git)
GitHub (Git)
Google Code Project Hosting (Git, Mercurial, Subversion)
Launchpad (Bazaar)
```
为了go get 能正常工作，你必须确保安装对应的源码管理工具，并把这些命令加入你的PATH中。其实go get支持自定义域名的功能，具体参见go help remote。

参数介绍：

* -d 只下载不安装
* -f 只有在你包含了-u参数的时候才有效，不让-u去验证import中的每一个都已经获取了，这对于本地fork的包特别有用
* -fix 在获取源码之后先运行fix，然后再去做其他的事情
* -t 同时也下载需要为运行测试所需要的包
* -u 强制使用网络去更新包和它的依赖包
* -v 显示执行的命令

### go install

这个命令在内部实际上分成了两步操作：
* 第一步: 生成结果文件(可执行文件或者.a包)
* 第二步: 把编译好的结果移到`$GOPATH/pkg`或者`$GOPATH/bin`。

参数支持go build的编译参数。只要记住一个参数`-v`就好了，这个随时随地查看底层的执行信息。

### go test

执行这个命令，会自动读取源码目录下面名为`*_test.go`的文件，生成并运行测试用的可执行文件。

默认情况下，不需要任何参数，它会自动把你源码包下面所有test文件测试完毕，当然你也可以带上参数，详情请参考`go help testflag`

这里介绍几个常用的参数：
* -bench regexp 执行相应的benchmarks，例如 -bench=.
* -cover 开启测试覆盖率
* -run regexp 只运行regexp匹配的函数，例如 -run=Array 那么就执行包含有Array开头的函数
* -v 显示测试的详细命令

### go tool

go tool下面聚集了很多命令，这里我们只介绍两个，fix和vet

1. `go tool fix .` 用来修复以前老版本代码到新版本，如go1之前老版本代码转化到go1,如API的变化
2. `go tool vet directory|files` 用来分析当前目录的代码是否都是正确的代码,例如是不是调用`fmt.Printf`里面的参数不正确，例如函数里面提前return了,然后出现了无用代码之类的。

### go generate

这个命令是从Go1.4开始才设计的，用于在编译前自动化生成某类代码。go generate和go build是完全不一样的命令，通过分析源码中特殊的注释，然后执行相应的命令。这些命令都是很明确的，没有任何的依赖在里面。而且大家在用这个之前心里面一定要有一个理念，这个go generate是给你用的，不是给使用你这个包的人用的，是方便你来生成一些代码的。

### godoc

go doc : 在Go1.2版本之前支持go doc命令
godoc : 之后全部移到了godoc这个命令

### 其他命令

go version 查看go当前的版本
go env 查看当前go的环境变量
go list 列出当前全部安装的package
go run 编译并运行Go程序